## BIG DATA NA LOGÍSTICA DO E-COMMERCE
### O uso de dados para otimizar o prazo de entrega ao cliente

## Pré-Processamento de Dados e Construção de Modelos de Machine Learning

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.12


In [2]:
!pip install -q -U watermark

In [3]:
# Imports
import joblib
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix
from sklearn.metrics import accuracy_score
%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "" --iversions

pandas    : 1.4.2
joblib    : 1.1.0
sklearn   : 1.1.2
seaborn   : 0.11.2
matplotlib: 3.5.1
numpy     : 1.21.5



### Carregando o Conjunto de dados

In [5]:
# Carrega os dados
dados = pd.read_csv('dados/dados.csv')

In [6]:
# Shape
dados.shape

(100581, 8)

In [7]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100581 entries, 0 to 100580
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           100581 non-null  int64  
 1   frete                100581 non-null  float64
 2   avaliação_cliente    100581 non-null  int64  
 3   peso_gr              100581 non-null  float64
 4   valor_pagamento      100581 non-null  float64
 5   tempo_processamento  100581 non-null  float64
 6   tempo_espera         100581 non-null  float64
 7   entregue_no_prazo    100581 non-null  int64  
dtypes: float64(5), int64(3)
memory usage: 6.1 MB


In [8]:
dados.drop(columns = ['Unnamed: 0'], inplace = True)

In [9]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100581 entries, 0 to 100580
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   frete                100581 non-null  float64
 1   avaliação_cliente    100581 non-null  int64  
 2   peso_gr              100581 non-null  float64
 3   valor_pagamento      100581 non-null  float64
 4   tempo_processamento  100581 non-null  float64
 5   tempo_espera         100581 non-null  float64
 6   entregue_no_prazo    100581 non-null  int64  
dtypes: float64(5), int64(2)
memory usage: 5.4 MB


In [10]:
dados.shape

(100581, 7)

### Divisão em Treino e Teste

In [11]:
dados.head()

,frete,avaliação_cliente,peso_gr,valor_pagamento,tempo_processamento,tempo_espera,entregue_no_prazo
0,13.63,4,1300.0,397.26,1.0,6.0,1
1,8.30,5,245.0,88.09,2.0,9.0,1
2,45.12,5,6550.0,194.12,3.0,13.0,0
3,42.85,5,7650.0,222.84,0.0,10.0,1
4,134.25,5,9850.0,1333.25,5.0,18.0,0


In [12]:
# Cria um objeto separado para a variável target
y = dados.entregue_no_prazo

In [13]:
# Cria um objeto separadado para as variáveis de entrada
X = dados.drop('entregue_no_prazo', axis = 1)

In [14]:
# Split em dados de treino e teste com amostragem estratificada
X_treino, X_teste, y_treino, y_teste = train_test_split(X, 
                                                        y, 
                                                        test_size = 0.25, 
                                                        random_state = 1234,
                                                        stratify = dados.entregue_no_prazo)

In [15]:
len(X_treino)

75435

In [16]:
len(X_teste)

25146

In [17]:
# Print do shape
print(X_treino.shape, X_teste.shape, y_treino.shape, y_teste.shape)

(75435, 6) (25146, 6) (75435,) (25146,)


In [18]:
X_treino.head(2)

,frete,avaliação_cliente,peso_gr,valor_pagamento,tempo_processamento,tempo_espera
54760,28.75,5,1900.0,64.75,1.0,17.0
25748,18.05,4,967.0,132.95,3.0,17.0


### Balanceamento de Classe

In [19]:
y_treino.value_counts()

1    40770
0    34665
Name: entregue_no_prazo, dtype: int64

In [20]:
# Carrega a função SMOTE
import imblearn
from imblearn.over_sampling import SMOTE

In [21]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "" --iversions

pandas    : 1.4.2
joblib    : 1.1.0
sklearn   : 1.1.2
imblearn  : 0.0
seaborn   : 0.11.2
matplotlib: 3.5.1
numpy     : 1.21.5



In [22]:
# Vamos aplicar a técnica de oversampling e aumentar o número de exemplos da classe minoritária
over_sampler = SMOTE(k_neighbors = 2)

#### O balanceamento de classe é feito somente com dados de treino.
* No caso dos dados de treino balancear a classe é importante, para que o modelo não aprenda mais sobre uma variável do que a outra.  

In [23]:
# Aplica o oversampling (deve ser feito somente com dados de treino)
X_res, y_res = over_sampler.fit_resample(X_treino, y_treino)

In [24]:
len(X_res)

81540

In [25]:
len(y_res)

81540

In [26]:
y_res.value_counts()

0    40770
1    40770
Name: entregue_no_prazo, dtype: int64

In [27]:
# Ajusta o nome do dataset de treino para X
X_treino = X_res

In [28]:
# Ajusta o nome do dataset de treino para y
y_treino = y_res

### Padronização dos Dados
* A Padronização significa transformar os dados de tal forma que eles tenham média zero e desvio padrão igual a 1. Portanto, aqui temos os dados  em escala de forma padronizada, de modo que a distribuição seja aproximadamente uma distribuição normal.

In [29]:
X_treino.head()

,frete,avaliação_cliente,peso_gr,valor_pagamento,tempo_processamento,tempo_espera
0,28.75,5,1900.0,64.75,1.0,17.0
1,18.05,4,967.0,132.95,3.0,17.0
2,16.86,5,2301.0,43.61,5.0,12.0
3,17.96,1,5550.0,196.46,3.0,9.0
4,16.79,1,150.0,46.78,1.0,29.0


In [30]:
# Calculamos média e desvio padrão dos dados de treino
treino_mean = X_treino.mean()
treino_std = X_treino.std()
print(treino_mean)
print(treino_std)

frete                    19.767978
avaliação_cliente         4.083615
peso_gr                1918.137028
valor_pagamento         154.025931
tempo_processamento       2.782105
tempo_espera             12.393125
dtype: float64
frete                    13.104737
avaliação_cliente         1.316671
peso_gr                3100.178679
valor_pagamento         199.723179
tempo_processamento       3.362634
tempo_espera              8.694482
dtype: float64


In [31]:
# Padronização
X_treino = (X_treino - treino_mean) / treino_std

In [32]:
X_treino.head()

,frete,avaliação_cliente,peso_gr,valor_pagamento,tempo_processamento,tempo_espera
0,0.685403,0.695986,-0.005850,-0.446998,-0.529973,0.529862
1,-0.131096,-0.063505,-0.306801,-0.105526,0.064799,0.529862
2,-0.221903,0.695986,0.123497,-0.552845,0.659571,-0.045215
3,-0.137964,-2.341978,1.171501,0.212464,0.064799,-0.390262
4,-0.227244,-2.341978,-0.570334,-0.536973,-0.529973,1.910048


In [33]:
# Describe
X_treino.describe()

,frete,avaliação_cliente,peso_gr,valor_pagamento,tempo_processamento,tempo_espera
count,8.154000e+04,8.154000e+04,8.154000e+04,8.154000e+04,8.154000e+04,8.154000e+04
mean,-7.339395e-14,1.292779e-14,-1.593129e-15,3.301928e-14,3.552738e-15,-6.336476e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.508461e+00,-2.341978e+00,-6.187182e-01,-7.711470e-01,-8.273587e-01,-1.425401e+00
25%,-4.828771e-01,-6.350515e-02,-5.219496e-01,-4.785420e-01,-5.299728e-01,-6.202929e-01
50%,-2.478146e-01,6.959859e-01,-3.929248e-01,-2.551328e-01,-2.325869e-01,-2.177387e-01
75%,1.207214e-01,6.959859e-01,-3.810652e-02,1.084249e-01,3.621849e-01,4.148465e-01
max,1.331213e+01,6.959859e-01,5.816395e+00,3.565362e+01,1.106808e+01,7.775837e+00


####  Usamos média e desvio dos dados de treino para padronizar os dados de teste.

* Padronizamos os dados de teste com o mesmo padrão usado em dados de treino, porque é assim que o modelo vai aprender. Quando usarmos o modelo treinado com novos dados, novamente aplicaremos a padronização.  Toda e qualquer transformação aplicada em treino tem que ser aplicada em teste e em novos dados.



In [34]:
# Usamos média e desvio de treino para padronizar o conjunto de dados de teste
X_teste = (X_teste - treino_mean) / treino_std

In [35]:
# Describe
X_teste.head()

,frete,avaliação_cliente,peso_gr,valor_pagamento,tempo_processamento,tempo_espera
10746,-0.441671,-0.063505,-0.473565,0.240103,0.659571,-0.390262
99442,1.100520,0.695986,1.639216,-0.004185,-0.232587,-0.965339
43048,-0.859077,-0.063505,-0.570334,0.222228,-0.232587,-0.620293
76574,-0.218087,0.695986,-0.538078,0.069066,-0.827359,-0.275246
71970,-0.270740,0.695986,-0.296156,-0.359527,0.659571,-0.275246


### Construção, Treinamento e Avaliação do Modelo 1 com Regressão Logística (Benchmark)
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

Para a primeira versão do modelo o ideal é escolher um algoritmo simples, fácil de compreender e que será usado como Benchmark.

Obs: Como parte do processo envolve aleatoriedade, os resultados podem ser ligeiramente diferentes a cada execução deste jupyter notebook.

In [36]:
# Define lista de hiperparâmetros
tuned_params_v1 = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 
                   'penalty': ['l1', 'l2']}

In [37]:
# Criaremos o modelo com GridSearch 
# Vários modelos serão criados com diferentes combinações de hiperparâmetros
modelo_v1 = GridSearchCV(LogisticRegression(), 
                         tuned_params_v1, 
                         scoring = 'roc_auc', 
                         n_jobs = -1)

In [38]:
# Treinamento do modelo
modelo_v1.fit(X_treino, y_treino)

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000,
                               10000],
                         'penalty': ['l1', 'l2']},
             scoring='roc_auc')

In [39]:
# Selecionamos o melhor modelo
modelo_v1.best_estimator_

LogisticRegression(C=0.1)

In [40]:
# Previsões com dados de teste
y_pred_v1 = modelo_v1.predict(X_teste)

In [41]:
# Mostra as 10 primeiras previsões
y_pred_v1[:10]

array([1, 1, 1, 1, 1, 1, 0, 0, 1, 0], dtype=int64)

In [42]:
# Obtemos as previsões no formato de probabilidade para cada classe
y_pred_proba_v1 = modelo_v1.predict_proba(X_teste)

In [43]:
# Mostra as 10 primeiras previsões
y_pred_proba_v1[:10]

array([[3.69977416e-02, 9.63002258e-01],
       [5.25886126e-07, 9.99999474e-01],
       [3.83595197e-04, 9.99616405e-01],
       [2.22612173e-01, 7.77387827e-01],
       [2.62344737e-01, 7.37655263e-01],
       [5.55727020e-10, 9.99999999e-01],
       [9.99597588e-01, 4.02412382e-04],
       [9.99994303e-01, 5.69740630e-06],
       [4.29648995e-05, 9.99957035e-01],
       [9.99999599e-01, 4.00822806e-07]])

In [44]:
# Obtemos as previsões no formato de probabilidade filtrando para a classe positiva
# Precisamos disso para calcula a Curva ROC
y_pred_proba_v1 = modelo_v1.predict_proba(X_teste)[:,1]

In [45]:
# Mostra as 10 primeiras previsões
y_pred_proba_v1[:10]

array([9.63002258e-01, 9.99999474e-01, 9.99616405e-01, 7.77387827e-01,
       7.37655263e-01, 9.99999999e-01, 4.02412382e-04, 5.69740630e-06,
       9.99957035e-01, 4.00822806e-07])

In [46]:
# Como exemplo, vamos verificar um dos pontos de dados (altere o valor de i se desejar)
i = 16 
print('Para o ponto de dado {}, classe real = {}, classe prevista = {}, probabilidade prevista = {}'.
      format(i, y_teste.iloc[i], y_pred_v1[i], y_pred_proba_v1[i]))

Para o ponto de dado 16, classe real = 0, classe prevista = 0, probabilidade prevista = 7.337541723325106e-12


In [47]:
# Matriz de confusão
confusion_matrix(y_teste, y_pred_v1)

array([[11556,     0],
       [    0, 13590]], dtype=int64)

In [48]:
# Extraindo cada valor da CM
tn, fp, fn, tp = confusion_matrix(y_teste, y_pred_v1).ravel()

In [49]:
print(tn, fp, fn, tp)

11556 0 0 13590


In [50]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v1 = roc_auc_score(y_teste, y_pred_v1)
print(roc_auc_v1)

1.0


In [51]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v1, tpr_v1, thresholds = roc_curve(y_teste, y_pred_proba_v1)

In [52]:
# AUC em teste
auc_v1 = auc(fpr_v1, tpr_v1)
print(auc_v1)

1.0


In [53]:
# Acurácia em teste
acuracia_v1 = accuracy_score(y_teste, y_pred_v1)
print(acuracia_v1)

1.0


### Feature Importance

In [54]:
# Construindo o modelo novamente com os melhores hiperparâmetros
# Isso é necessário pois a versão final não deve ter o GridSearchCV
modelo_v1 = LogisticRegression(C = 1)
modelo_v1.fit(X_treino, y_treino)

LogisticRegression(C=1)

In [55]:
# Obtemos os coeficientes pelo maior maior usando np.argsort
indices = np.argsort(-abs(modelo_v1.coef_[0,:]))

In [56]:
print("Variáveis mais importantes para o resultado do modelo_v1:")
print(50*'-')
for feature in X.columns[indices]:
    print(feature)

Variáveis mais importantes para o resultado do modelo_v1:
--------------------------------------------------
tempo_espera
tempo_processamento
frete
avaliação_cliente
valor_pagamento
peso_gr


In [57]:
# Salva o modelo em disco
with open('modelos/modelo_v1.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v1, 'modelos/modelo_v1.pkl') 

In [58]:
# Cria um dataframe para receber as métricas de cada modelo
df_modelos = pd.DataFrame()

In [59]:
# Dicionário com as métricas do modelo_v1
dict_modelo_v1 = {'Nome': 'modelo_v1', 
                  'Algoritmo': 'Regressão Logística', 
                  'ROC_AUC Score': roc_auc_v1,
                  'AUC Score': auc_v1,
                  'Acurácia': acuracia_v1}

In [60]:
# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v1, ignore_index = True)

In [61]:
display(df_modelos)

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia
0,modelo_v1,Regressão Logística,1.0,1.0,1.0


### Construção, Treinamento e Avaliação do Modelo 2 com Random Forest
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Vamos tentar o algoritmo Random Forest.

In [62]:
# Grid de hiperparâmetros
tuned_params_v2 = {'n_estimators': [100, 200, 300, 400, 500], 
                   'min_samples_split': [2, 5, 10], 
                   'min_samples_leaf': [1, 2, 4]}

In [63]:
# Cria o modelo com RandomizedSearchCV para buscar a melhor combinação de hiperparâmetros
modelo_v2 = RandomizedSearchCV(RandomForestClassifier(), 
                               tuned_params_v2, 
                               n_iter = 15, 
                               scoring = 'roc_auc', 
                               n_jobs  = -1)

In [64]:
# Treina o modelo
modelo_v2.fit(X_treino, y_treino)

RandomizedSearchCV(estimator=RandomForestClassifier(), n_iter=15, n_jobs=-1,
                   param_distributions={'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   scoring='roc_auc')

In [65]:
# Extrai o melhor modelo
modelo_v2.best_estimator_

RandomForestClassifier(min_samples_leaf=4, min_samples_split=5,
                       n_estimators=500)

In [66]:
# Previsões em teste
y_pred_v2 = modelo_v2.predict(X_teste)

In [67]:
# Obtém as previsões para a classe positiva
y_pred_proba_v2 = modelo_v2.predict_proba(X_teste)[:,1]

In [68]:
# Matriz de Confusão
confusion_matrix(y_teste, y_pred_v2)

array([[11556,     0],
       [    0, 13590]], dtype=int64)

In [69]:
# Curva ROC nos dados e previsões em teste
roc_auc_v2 = roc_auc_score(y_teste, y_pred_v2)
print(roc_auc_v2)

1.0


In [70]:
# Curva ROC nos dados e previsões em teste
fpr_v2, tpr_v2, thresholds = roc_curve(y_teste, y_pred_proba_v2)

In [71]:
# AUC em teste
auc_v2 = auc(fpr_v2, tpr_v2)
print(auc_v2)

1.0


In [72]:
# Acurácia em teste
acuracia_v2 = accuracy_score(y_teste, y_pred_v2)
print(acuracia_v2)

1.0


### Feature Importance

In [73]:
# Recria o modelo com os melhores hiperparâmetros
modelo_v2 = RandomForestClassifier(n_estimators = 200, min_samples_split = 5, min_samples_leaf = 4)
modelo_v2.fit(X_treino, y_treino)

RandomForestClassifier(min_samples_leaf=4, min_samples_split=5,
                       n_estimators=200)

In [74]:
# Variáveis mais relevantes
indices = np.argsort(-modelo_v2.feature_importances_)
print("Variáveis mais importantes para o resultado do modelo_v2:")
print(50*'-')
for feature in X.columns[indices]:
    print(feature)

Variáveis mais importantes para o resultado do modelo_v2:
--------------------------------------------------
tempo_espera
frete
tempo_processamento
avaliação_cliente
valor_pagamento
peso_gr


In [75]:
# Salva o modelo em disco
with open('modelos/modelo_v2.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v2, 'modelos/modelo_v2.pkl') 

In [76]:
# Dicionário com as métricas do modelo_v2
dict_modelo_v2 = {'Nome': 'modelo_v2', 
                  'Algoritmo': 'Random Forest', 
                  'ROC_AUC Score': roc_auc_v2,
                  'AUC Score': auc_v2,
                  'Acurácia': acuracia_v2}

In [77]:
# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v2, ignore_index = True)

In [78]:
display(df_modelos)

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia
0,modelo_v1,Regressão Logística,1.0,1.0,1.0
1,modelo_v2,Random Forest,1.0,1.0,1.0


### Construção, Treinamento e Avaliação do Modelo 3 com KNN
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

Para esse algoritmo precisamos antes definir o valor de K, que é o número de vizinhos mais próximos.

In [79]:
# Lista de possíveis valores de K
vizinhos = list(range(1, 20, 2))

In [80]:
# Lista para os scores
cv_scores = []

In [81]:
# Validação cruzada para determinar o melhor valor de k
for k in vizinhos:
    knn = KNeighborsClassifier(n_neighbors = k)
    scores = cross_val_score(knn, X_treino, y_treino, cv = 5, scoring = 'accuracy')
    cv_scores.append(scores.mean()) 

In [82]:
# Ajustando o erro de classificação
erro = [1 - x for x in cv_scores]

In [83]:
# Determinando o melhor valor de k (com menor erro)
optimal_k = vizinhos[erro.index(min(erro))]
print('O valor ideal de k é %d' % optimal_k)

O valor ideal de k é 15


In [84]:
# Criamos o modelo versão 3
modelo_v3 = KNeighborsClassifier(n_neighbors = optimal_k)

In [85]:
# Treinamento
modelo_v3.fit(X_treino, y_treino)

KNeighborsClassifier(n_neighbors=15)

In [86]:
# Previsões
y_pred_v3 = modelo_v3.predict(X_teste)

In [87]:
# Confusion Matrix
confusion_matrix(y_teste, y_pred_v3)

array([[11321,   235],
       [  268, 13322]], dtype=int64)

In [88]:
# Previsão de probabilidade da classe positiva
y_pred_proba_v3 = modelo_v3.predict_proba(X_teste)[:,1]

In [89]:
# Calcula ROC_AUC em teste
roc_auc_v3 = roc_auc_score(y_teste, y_pred_v3)
print(roc_auc_v3)

0.9799719305243226


In [90]:
# Calcula curva ROC
fpr_v3, tpr_v3, thresholds = roc_curve(y_teste, y_pred_proba_v3)

In [91]:
# Calcula AUC em teste
auc_v3 = auc(fpr_v3, tpr_v3)
print(auc_v3)

0.9984589805639162


In [92]:
# Calcula acurácia
acuracia_v3 = accuracy_score(y_teste, y_pred_v3)
print(acuracia_v3)

0.9799968185794957


#### Obs: Com o algoritmo KNN não extraímos as variáveis mais importantes, pois o conceito do algoritmo é diferente.

In [93]:
# Salva o modelo em disco
with open('modelos/modelo_v3.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v3, 'modelos/modelo_v3.pkl') 

In [94]:
# Dicionário com as métricas do modelo_v3
dict_modelo_v3 = {'Nome': 'modelo_v3', 
                  'Algoritmo': 'KNN', 
                  'ROC_AUC Score': roc_auc_v3,
                  'AUC Score': auc_v3,
                  'Acurácia': acuracia_v3}

In [95]:
# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v3, ignore_index = True)

In [96]:
display(df_modelos)

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia
0,modelo_v1,Regressão Logística,1.000000,1.000000,1.000000
1,modelo_v2,Random Forest,1.000000,1.000000,1.000000
2,modelo_v3,KNN,0.979972,0.998459,0.979997


### Construção, Treinamento e Avaliação do Modelo 4 com Decision Tree
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

Na versão 4 do modelo usaremos um modelo de árvore de decisão.

In [97]:
# Hiperparâmetros
tuned_params_v4 = {'min_samples_split': [2, 3, 4, 5, 7], 
                   'min_samples_leaf': [1, 2, 3, 4, 6], 
                   'max_depth': [2, 3, 4, 5, 6, 7]}

In [98]:
# Cria o modelo com RandomizedSearchCV
modelo_v4 = RandomizedSearchCV(DecisionTreeClassifier(), 
                               tuned_params_v4, 
                               n_iter = 15, 
                               scoring = 'roc_auc', 
                               n_jobs = -1)

In [99]:
# Treinamento
modelo_v4.fit(X_treino, y_treino)

RandomizedSearchCV(estimator=DecisionTreeClassifier(), n_iter=15, n_jobs=-1,
                   param_distributions={'max_depth': [2, 3, 4, 5, 6, 7],
                                        'min_samples_leaf': [1, 2, 3, 4, 6],
                                        'min_samples_split': [2, 3, 4, 5, 7]},
                   scoring='roc_auc')

In [100]:
# Melhor modelo
modelo_v4.best_estimator_

DecisionTreeClassifier(max_depth=2, min_samples_leaf=2, min_samples_split=7)

In [101]:
# Previsões de classe
y_pred_v4 = modelo_v4.predict(X_teste)

In [102]:
# Previsões de probabilidade
y_pred_proba_v4 = modelo_v4.predict_proba(X_teste)[:,1]

In [103]:
# Confusion matrix
confusion_matrix(y_teste, y_pred_v4)

array([[11556,     0],
       [    0, 13590]], dtype=int64)

In [104]:
# Calcula ROC AUC score
roc_auc_v4 = roc_auc_score(y_teste, y_pred_v4)
print(roc_auc_v4)

1.0


In [105]:
# Curva ROC
fpr_v4, tpr_v4, thresholds = roc_curve(y_teste, y_pred_proba_v4)

In [106]:
# AUC
auc_v4 = auc(fpr_v4, tpr_v4)
print(auc_v4)

1.0


In [107]:
# Calcula acurácia
acuracia_v4 = accuracy_score(y_teste, y_pred_v4)
print(acuracia_v4)

1.0


### Feature Importance

In [108]:
# Recriando o modelo
modelo_v4 = DecisionTreeClassifier(min_samples_split = 2, min_samples_leaf = 6, max_depth = 4)
modelo_v4.fit(X_treino, y_treino)

DecisionTreeClassifier(max_depth=4, min_samples_leaf=6)

In [109]:
# Variáveis mais importantes
indices = np.argsort(-modelo_v4.feature_importances_)
print("Variáveis mais importantes para o resultado do modelo_v4:")
print(50*'-')
for feature in X.columns[indices]:
    print(feature)

Variáveis mais importantes para o resultado do modelo_v4:
--------------------------------------------------
tempo_espera
frete
avaliação_cliente
peso_gr
valor_pagamento
tempo_processamento


In [110]:
# Salva o modelo em disco
with open('modelos/modelo_v4.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v4, 'modelos/modelo_v4.pkl') 

In [111]:
# Dicionário com as métricas do modelo_v4
dict_modelo_v4 = {'Nome': 'modelo_v4', 
                  'Algoritmo': 'Decision Tree', 
                  'ROC_AUC Score': roc_auc_v4,
                  'AUC Score': auc_v4,
                  'Acurácia': acuracia_v4}

In [112]:
# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v4, ignore_index = True)

In [113]:
display(df_modelos)

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia
0,modelo_v1,Regressão Logística,1.000000,1.000000,1.000000
1,modelo_v2,Random Forest,1.000000,1.000000,1.000000
2,modelo_v3,KNN,0.979972,0.998459,0.979997
3,modelo_v4,Decision Tree,1.000000,1.000000,1.000000


### Seleção do Melhor Modelo

In [114]:
# Usaremos o modelo que teve o maior AUC Score, por se tratar de uma métrica global
# O score AUC é o ideal para comparar modelos de diferentes algoritmos
df_melhor_modelo = df_modelos[df_modelos['AUC Score'] == df_modelos['AUC Score'].max()]

In [115]:
df_melhor_modelo

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia
0,modelo_v1,Regressão Logística,1.0,1.0,1.0
1,modelo_v2,Random Forest,1.0,1.0,1.0
3,modelo_v4,Decision Tree,1.0,1.0,1.0


## Previsões com o Melhor Modelo Treinado

In [116]:
# Obtemos o nome do melhor modelo
modelo = df_melhor_modelo.Nome.to_string(index = False)
modelo

'modelo_v1\nmodelo_v2\nmodelo_v4'

In [117]:
# Carregamos os melhores modelos do disco
melhor_modelo = joblib.load('modelos/modelo_v1.pkl')
melhor_modelo

LogisticRegression(C=1)

In [118]:
dados.columns

Index(['frete', 'avaliação_cliente', 'peso_gr', 'valor_pagamento',
       'tempo_processamento', 'tempo_espera', 'entregue_no_prazo'],
      dtype='object')

In [119]:
# Dados brutos de um novo pedido
# O número de colunas deve ser o mesmo do que foi usado em treino
novo_pedido = [21.70, 3, 320.0, 97.25, 2.0, 13.0]

In [120]:
# Converte o objeto para array
arr_pedido = np.array(novo_pedido)

In [121]:
# Usamos média e desvio de treino para padronizar novos dados
arr_pedido = (arr_pedido - treino_mean) / treino_std

In [122]:
# Converte o objeto para array
arr_pedido = np.array(arr_pedido)

In [123]:
# Dados do pedido padronizados (exatamente como o modelo espera receber os dados)
arr_pedido

array([ 0.1474293 , -0.82299615, -0.51549836, -0.28427312, -0.23258691,
        0.06980004])

In [124]:
# Previsões de classe (por que a linha abaixo apresenta erro?)
#pred_novo_pedido = melhor_modelo.predict(arr_pedido)

In [125]:
# Previsões de classe
pred_novo_pedido = melhor_modelo.predict(arr_pedido.reshape(1, -1))

In [126]:
# Verifica o valor e imprime o resultado final
if pred_novo_pedido == 1:
    print('Este pedido será entregue no prazo!')
else:
    print('Este pedido não será entregue no prazo!')

Este pedido não será entregue no prazo!


In [127]:
# Carregamos os melhores modelos do disco
melhor_modelo = joblib.load('modelos/modelo_v4.pkl')
melhor_modelo

DecisionTreeClassifier(max_depth=4, min_samples_leaf=6)

In [128]:
# Dados brutos de um novo pedido
# O número de colunas deve ser o mesmo do que foi usado em treino
novo_pedido = [31.50, 4, 480.0, 65.10, 2.0, 8.0]

In [129]:
# Converte o objeto para array
arr_pedido = np.array(novo_pedido)

In [130]:
# Usamos média e desvio de treino para padronizar novos dados
arr_pedido = (arr_pedido - treino_mean) / treino_std

In [131]:
# Converte o objeto para array
arr_pedido = np.array(arr_pedido)

In [132]:
# Dados do pedido padronizados (exatamente como o modelo espera receber os dados)
arr_pedido

array([ 0.89525051, -0.06350515, -0.46388843, -0.44524592, -0.23258691,
       -0.50527737])

In [133]:
# Previsões de classe
pred_novo_pedido = melhor_modelo.predict(arr_pedido.reshape(1, -1))

In [134]:
# Verifica o valor e imprime o resultado final
if pred_novo_pedido == 1:
    print('Este pedido será entregue no prazo!')
else:
    print('Este pedido não será entregue no prazo!')

Este pedido será entregue no prazo!


In [135]:
# Carregamos os melhores modelos do disco
melhor_modelo = joblib.load('modelos/modelo_v2.pkl')
melhor_modelo

RandomForestClassifier(min_samples_leaf=4, min_samples_split=5,
                       n_estimators=200)

In [136]:
dados.columns

Index(['frete', 'avaliação_cliente', 'peso_gr', 'valor_pagamento',
       'tempo_processamento', 'tempo_espera', 'entregue_no_prazo'],
      dtype='object')

In [137]:
# Dados brutos de um novo pedido
# O número de colunas deve ser o mesmo do que foi usado em treino
novo_pedido = [18.95, 5, 200.0, 102.00, 1.0, 7.0]

In [138]:
# Converte o objeto para array
arr_pedido = np.array(novo_pedido)

In [139]:
# Usamos média e desvio de treino para padronizar novos dados
arr_pedido = (arr_pedido - treino_mean) / treino_std

In [140]:
# Converte o objeto para array
arr_pedido = np.array(arr_pedido)

In [141]:
# Dados do pedido padronizados (exatamente como o modelo espera receber os dados)
arr_pedido

array([-0.06241848,  0.69598585, -0.55420581, -0.2604902 , -0.5299728 ,
       -0.62029285])

In [142]:
# Previsões de classe
pred_novo_pedido = melhor_modelo.predict(arr_pedido.reshape(1, -1))

In [143]:
# Verifica o valor e imprime o resultado final
if pred_novo_pedido == 1:
    print('Este pedido será entregue no prazo!')
else:
    print('Este pedido não será entregue no prazo!')

Este pedido será entregue no prazo!
